# SUM and COUNT

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app05")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@756efacf

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@3311df06

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val world = hiveCxt.table("sqlzoo.world")

world: DataFrame = [name: string, continent: string ... 6 more fields]

## 1. Total world population

Show the total population of the world.

```
world(name, continent, area, population, gdp)
```

In [5]:
world.agg(sum($"population")).showHTML()

sum(population)
7.615648652E9


## 2. List of continents

List all the continents - just once each.

In [6]:
world.select("continent").distinct().showHTML()

continent
Europe
Eurasia
Africa
North America
Caribbean
South America
"Federated States of,Oceania"
Oceania
Asia


## 3. GDP of Africa

Give the total GDP of Africa

In [7]:
world.filter($"continent"==="Africa").agg(sum("gdp")).showHTML()

sum(gdp)
1.964824E12


## 4. Count the big countries

How many countries have an **area** of at least 1000000

In [8]:
world.filter($"area">1e6).agg(count("name")).showHTML()

count(name)
29


## 5. Baltic states population

What is the total **population** of ('Estonia', 'Latvia', 'Lithuania')

In [9]:
(world.filter($"name".isin(List("Estonia", "Latvia", "Lithuania"): _*))
 .agg(sum("population")).showHTML())

sum(population)
6028631.0


## 6. Counting the countries of each continent

For each **continent** show the **continent** and number of countries.

> _Using GROUP BY and HAVING_   
> You may want to look at these examples: [Using GROUP BY and HAVING](https://sqlzoo.net/wiki/Using_GROUP_BY_and_HAVING.).

In [10]:
world.groupBy("continent").count().showHTML()

continent,count
Europe,44
Eurasia,2
Africa,53
North America,11
Caribbean,11
South America,13
"Federated States of,Oceania",1
Oceania,13
Asia,47


## 7. Counting big countries in each continent

For each **continent** show the **continent** and number of countries with populations of at least 10 million.

In [11]:
(world.filter($"population" >= 1e7)
 .select("continent", "name")
 .groupBy("continent")
 .count().showHTML())

continent,count
Europe,15
Eurasia,1
Africa,31
North America,4
Caribbean,3
South America,8
Oceania,1
Asia,28


## 8. Counting big continents

List the continents that **have** a total population of at least 100 million.

In [12]:
(world.groupBy("continent").agg(sum("population"))
 .filter($"sum(population)" >= 1e8)
 .select("continent").showHTML())

continent
Europe
Eurasia
Africa
North America
South America
Asia


In [13]:
spark.stop()